In [7]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("FRED_API_KEY")
# api_key = os.getenv("FRED_API_KEY")

if api_key is None:
    raise ValueError("FRED_API_KEY environment variable not set.")

In [16]:
# Define the function to fetch data from FRED API for a list of series IDs
def fetch_fred_data(series_id, api_key):
    base_url = f"https://api.stlouisfed.org/fred/series/observations"
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json'
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()['observations']
        df = pd.DataFrame(data)
        # When downloading from the website, the data is saved as: 
        # DATE | series_id 
        df['DATE'] = pd.to_datetime(df['date'])
        df[f'{series_id}'] = pd.to_numeric(df['value'], errors='coerce')
        return df[['DATE', f'{series_id}']].copy()
    else:
        print(f"Error fetching data for {series_id}: {response.status_code}")
        return None
    
def compare_and_update_data(series_id, new_data, data_path):
    file_path = os.path.join(data_path, f"{series_id}.csv")

    # Check if the file exists
    if os.path.exists(file_path):
        # Load existing data
        existing_data = pd.read_csv(file_path)
        existing_data['DATE'] = pd.to_datetime(existing_data['DATE'])

        # Find the most recent date in the existing data
        last_existing_date = existing_data['DATE'].max()

        # Compare with the latest data from FRED
        last_new_date = new_data['DATE'].max()

        if last_new_date > last_existing_date:
            print(f"New data found for {series_id}. Updating file.")
            # Save the new data to the CSV file, replacing the old file
            new_data.to_csv(file_path, index=False)
        else:
            # print(f"No new data for {series_id}.")
            pass
    else:
        # If file doesn't exist, save the new data
        print(f"No existing data for {series_id}. Creating new file.")
        new_data.to_csv(file_path, index=False)

# Main function to process a list of series IDs
def process_series_ids(series_ids, api_key, data_path="./data"):
    for series_id in series_ids:
        # Fetch data for each series ID
        new_data = fetch_fred_data(series_id, api_key)
        if new_data is not None:
            # Compare and update data files
            compare_and_update_data(series_id, new_data, data_path)

In [20]:
series_ids = [
    "ACTLISCOUUS",
    "CSUSHPINSA",
    "HOUST",
    "MEDDAYONMARUS",
    "MEHOINUSA672N",
    "MORTGAGE30US",
    "MSACSR",
    "MSPUS",
    "NEWLISCOUUS",
    "T10Y3M",
    "UNRATE",
]

process_series_ids(series_ids, api_key)

No new data for ACTLISCOUUS. File is up to date.
No new data for CSUSHPINSA. File is up to date.
No new data for HOUST. File is up to date.
No new data for MEDDAYONMARUS. File is up to date.
No new data for MEHOINUSA672N. File is up to date.
New data found for MORTGAGE30US. Updating file.
No new data for MSACSR. File is up to date.
No new data for MSPUS. File is up to date.
No new data for NEWLISCOUUS. File is up to date.
New data found for T10Y3M. Updating file.
New data found for UNRATE. Updating file.
